In [ ]:
!pip install pyhht

In [ ]:
!pip install lightgbm

In [1]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [1]:
import tensorflow as tf
import matplotlib
# Force matplotlib to not use any Xwindows backend.
matplotlib.use('Agg')
# import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from keras.models import Sequential, model_from_json, load_model
from keras.layers.core import Dense, Dropout, Flatten, Activation, SpatialDropout2D, Reshape, Lambda
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU, PReLU, LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard
import librosa
import librosa.display

import pdb
import os
from os.path import join as ojoin
import time  
import numpy as np
import numpy.matlib
import argparse
import random

from pyhht.visualization import plot_imfs
import IPython.display as ipd

Using TensorFlow backend.


In [3]:
os.getcwd()

'D:\\jupyter_notebook\\denoise'

In [4]:
def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            if filename.endswith('.wav'):
            # Join the two strings in order to form the full filepath.
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)  # Add it to the list.
                # pdb.set_trace()
    return file_paths

In [5]:
# fix the order of datas and labels
mixed_file = sorted(get_filepaths('mixed/mixed_all_snr/'))
cleaned_file = sorted(get_filepaths('processed_data/'))
len(mixed_file), len(cleaned_file), mixed_file[2010:2020], cleaned_file[2010:2020]    # check datas corresponding to labels

(5760,
 5760,
 ['mixed/mixed_all_snr/hsiaoen_T10-10_20200622_toll_bar_snr6.wav',
  'mixed/mixed_all_snr/hsiaoen_T10-1_20200622_fall_streets1_snr-10.wav',
  'mixed/mixed_all_snr/hsiaoen_T10-2_20200622_summer_beach2_snr-6.wav',
  'mixed/mixed_all_snr/hsiaoen_T10-3_20200622_brook_in_a_cave_snr-11.wav',
  'mixed/mixed_all_snr/hsiaoen_T10-4_20200622_washing_machine2_snr-5.wav',
  'mixed/mixed_all_snr/hsiaoen_T10-5_20200622_strong_wind1_snr-5.wav',
  'mixed/mixed_all_snr/hsiaoen_T10-6_20200622_brook_in_a_cave_snr-11.wav',
  'mixed/mixed_all_snr/hsiaoen_T10-7_20200622_ountain_children_snr4.wav',
  'mixed/mixed_all_snr/hsiaoen_T10-8_20200622_ort1_snr-3.wav',
  'mixed/mixed_all_snr/hsiaoen_T10-9_20200622_summer_beach1_snr5.wav'],
 ['processed_data/hsiaoen\\hsiaoen_T10-10_20200622.wav',
  'processed_data/hsiaoen\\hsiaoen_T10-1_20200622.wav',
  'processed_data/hsiaoen\\hsiaoen_T10-2_20200622.wav',
  'processed_data/hsiaoen\\hsiaoen_T10-3_20200622.wav',
  'processed_data/hsiaoen\\hsiaoen_T10-4_202

In [25]:
# let datas separated into training and testing
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(mixed_file, cleaned_file, test_size=0.3, random_state=42)

In [26]:
Train_Noisy_lists=X_train
Train_Clean_paths= y_train

Val_Noisy_lists  = X_val
Val_Clean_paths = y_val
          
Num_valdata=len(Val_Noisy_lists)   
Num_traindata=len(Train_Noisy_lists)

# check the length of training datas and validation
len(Train_Noisy_lists), len(Train_Clean_paths), len(Val_Noisy_lists), len(Val_Clean_paths)

(4032, 4032, 1728, 1728)

In [28]:
# find the max length of voice data
max_len = 0
for data in Train_Noisy_lists:
    noisy, rate = librosa.load(data, sr=16000, mono=True)
    if noisy.shape[0] > max_len:
        max_len = noisy.shape[0]
max_len

88800

In [29]:
noisy_stft_list = []
for data in Train_Noisy_lists:
    noisy, rate = librosa.load(data, sr=16000, mono=True, dtype='float32')
    noisy = np.hstack((noisy, np.zeros(max_len - noisy.shape[0])))
    noisy = librosa.stft(noisy, n_fft=512, hop_length=256, win_length=512, center=False)
    noisy0 = np.abs(noisy)
    noisy_stft_list += [noisy0]

clean_stft_list = []
for data in Train_Clean_paths:
    noisy, rate = librosa.load(data, sr=16000, mono=True, dtype='float32')
    noisy = np.hstack((noisy, np.zeros(max_len - noisy.shape[0])))
    noisy = librosa.stft(noisy, n_fft=512, hop_length=256, win_length=512, center=False)
    noisy0 = np.abs(noisy)
    noisy_stft_list += [noisy0]
len(noisy_stft_list)

4032

In [ ]:
def train_data_generator(noisy_list, clean_path):
    index=0
    while True:
        noisy, rate = librosa.load(data, sr=16000, mono=True, dtype='float32')
        noisy = np.hstack((noisy, np.zeros(max_len - noisy.shape[0])))
        noisy = librosa.stft(noisy, n_fft=512, hop_length=256, win_length=512, center=False)
        noisy0 = np.abs(noisy)
        noisy_stft_list += [noisy0]
        noisy = noisy.astype('float32')
        noisy = scipy.signal.stft(noisy, nperseg=512, nfft=512, fs=16000, noverlap=256)
        noisy0 = np.abs(noisy)
        noisy0 = np.reshape(noisy0,(1, np.shape(noisy0)[0], np.shape(noisy0)[1],1))
        
        rate, clean = wavfile.read(clean_path[index])
        clean = clean.astype('float32')
        sr, _, clean = scipy.signal.stft(clean, nperseg=512, nfft=512, fs=16000, noverlap=256)
        clean0 = np.abs(clean)
        clean0 = np.reshape(clean0,(1, np.shape(clean0)[0], np.shape(clean0)[1],1))

        index += 1
        if index == len(noisy_list):
            index = 0
        yield noisy0, clean0

def val_data_generator(noisy_list, clean_path):
    index=0
    while True:
         #noisy, rate  = librosa.load(noisy_list[index],sr=16000)       
        rate, noisy = wavfile.read(noisy_list[index])
        noisy = noisy.astype('float32')
        sr, _, noisy = scipy.signal.stft(noisy, nperseg=512, nfft=512, fs=16000, noverlap=256)
        noisy0 = np.abs(noisy)
        noisy0 = np.reshape(noisy0,(1, np.shape(noisy0)[0], np.shape(noisy0)[1],1))
          
        rate, clean = wavfile.read(clean_path[index])
        clean = clean.astype('float32')
        sr, _, clean = scipy.signal.stft(clean, nperseg=512, nfft=512, fs=16000, noverlap=256)
        clean0 = np.abs(clean)
        clean0 = np.reshape(clean0,(1, np.shape(clean0)[0], np.shape(clean0)[1],1))

        index += 1
        if index == len(noisy_list):
            index = 0
        yield noisy0, clean0

In [ ]:
start_time = time.time()

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation ='relu', input_shape = (257, None, 1), kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'Same', activation ='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
# model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = 'Same', activation ='relu', kernel_initializer='he_normal'))
# model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = 'Same', activation ='relu', kernel_initializer='he_normal'))
model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = 'Same', activation ='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = 'Same', activation ='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same', activation ='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 1, kernel_size = (5,5),padding = 'Same', activation ='tanh', kernel_initializer='he_normal'))

model.summary()
print('finished!!!')

In [ ]:
epoch = 5
batch_size = 1
model.compile(loss='mse', optimizer='adam')

with open('{}.json'.format('firsttry'),'w') as f:    # save the model
    f.write(model.to_json()) 
checkpointer = ModelCheckpoint(filepath='./model_h5/{}.hdf5'.format('stft3_0714'), verbose=1, save_best_only=True, mode='min')  

print ('training...')

g1 = train_data_generator(Train_Noisy_lists, Train_Clean_paths)
g2 = val_data_generator(Test_Noisy_lists, Test_Clean_paths)

tbCallBack = TensorBoard(log_dir='./logs',  # log 目錄
                 histogram_freq=0,  # 按照何等频率（epoch）來計算直方圖，0為不計算
                 batch_size=32,     # 用多大量的數據計算直方圖
                 write_graph=True,  # 是否存储網路結構圖
                 write_grads=True, # 是否可視化梯度直方圖
                 write_images=True,# 是否可視化參數
                 embeddings_freq=0,
                 embeddings_layer_names=None,
                 embeddings_metadata=None)
hist=model.fit_generator(g1,
                         samples_per_epoch=Num_traindata,
                        # samples_per_epoch=50,
                        nb_epoch=epoch,
                        verbose=1,
                         validation_data=g2,
                         nb_val_samples=Num_testdata,
                         max_q_size=1,
                         nb_worker=1,
                         pickle_safe=False,
                         )

In [ ]:
# plotting the learning curve
TrainERR=hist.history['loss']
ValidERR=hist.history['val_loss']
print ('@%f, Minimun error:%f, at iteration: %i' % (hist.history['val_loss'][epoch-1], np.min(np.asarray(ValidERR)),np.argmin(np.asarray(ValidERR))+1))
# print 'drawing the training process...'
plt.clf()
plt.figure(4)
plt.plot(range(1,epoch+1),TrainERR,'b',label='TrainERR')
plt.plot(range(1,epoch+1),ValidERR,'r',label='ValidERR')
plt.xlim([1,epoch])
plt.legend()
plt.xlabel('epoch')
plt.ylabel('error')
plt.grid(True)
plt.show()
plt.savefig('Learning_curve_{}7.png'.format('FCN_firsttry'), dpi=150)

end_time = time.time()

In [ ]:
# predict the top ten datas
for path in Test_Noisy_lists[:10]:
    S = path.split('/')
    wave_name = S[2]
    rate, noisy = wavfile.read(path)
    noisy = noisy.astype('float32')
    sr, _, noisy = scipy.signal.stft(noisy, nperseg=512, nfft=512, fs=16000, noverlap=256)
    noisy0 = np.abs(noisy)
    n_phase = np.angle(noisy)

    noisy0 = np.reshape(noisy0, (1, np.shape(noisy0)[0], np.shape(noisy0)[1], 1))
    enhanced = np.squeeze(model.predict(noisy0, verbose=0, batch_size=batch_size))
    Rec = np.multiply(enhanced, np.exp(1j * n_phase))
    _, enhanced = scipy.signal.istft(Rec, nperseg=512,nfft=512,fs=16000,noverlap=256)
    enhanced = enhanced * maxv

    librosa.output.write_wav(os.path.join("predict_result/test7_0713/", wave_name), (enhanced).astype(np.int16), 16000)